# Cross-Lingual Alignment (English ↔ Hindi)

## Preprocess Hindi Corpus
You can use the indic-nlp-library for Hindi-specific tokenization and normalization.

In [ ]:
import os
os.environ["INDIC_RESOURCES_PATH"] = "path_to/indic_nlp_resources"

In [ ]:
from indicnlp import common
from indicnlp import loader
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from collections import Counter
import numpy as np

# Set up Indic NLP resources
INDIC_RESOURCES_PATH = "path_to/indic_nlp_resources"  # Replace with your path
common.set_resources_path(INDIC_RESOURCES_PATH)
loader.load()

# Load Hindi corpus file
path = "D:\\RESEARCH related\\PreCog tasks\\Language_representations\\Data\\hin_news_2020_300K\\hin_news_2020_300K-sentences.txt"
with open(path, 'r', encoding='utf-8') as f:
    hindi_lines = f.readlines()

# Preprocess Hindi text
def preprocess_hindi(text):
    normalizer = IndicNormalizerFactory().get_normalizer("hi")
    text = normalizer.normalize(text)
    tokens = list(indic_tokenize.trivial_tokenize(text, lang='hi'))
    return tokens

# Example usage
sample_text = hindi_lines[0]
tokens = preprocess_hindi(sample_text)
print(tokens)

<>:9: SyntaxWarning: invalid escape sequence '\R'
<>:9: SyntaxWarning: invalid escape sequence '\R'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13044\660233377.py:9: SyntaxWarning: invalid escape sequence '\R'
  path = "D:\RESEARCH related\PreCog tasks\Language_representations\Data\hin_news_2020_300K\hin_news_2020_300K-sentences.txt"


ModuleNotFoundError: No module named 'indicnlp'